# Use the Kalman filter and OpenBB for parameterless indicators

The Kalman filter is an algorithm that tracks an object in state space. Given a sequence of noisy measurements, the Kalman Filter recovers the “true state” of the object. In today’s newsletter, you’ll recover the “true average” of a series of stock prices.

The Kalman filter updates its estimates at every time step so it’s useful for estimating rolling parameters. Quants use it to smooth trading indicators on noisy price data to signal trading opportunities. The Kalman filter is an unsupervised learning algorithm so you don’t need to select a window length. This helps reduce the risk of overfitting-though does not remove it.

Use the Kalman filter to smooth rolling indicators

The Kalman filter is useful for computing the moving average or for smoothing out estimates of other quantities. For example, if you already computed the moving Sharpe ratio, you can smooth it using a Kalman filter. At each time step, the algorithm estimates the current state of the system using the transition matrix, takes in new measurements, then updates the estimated current state.

This notebook shows how to:

1 .Get stock price data with the OpenBB SDK
2. Build a moving average with the Kalman filter

In [4]:
# Step 1: Get stock price data with the OpenBB SDK

# Start by importing the libraries you need. pykalman is the “dead-simple” Kalman filter library for Python.

import pandas as pd
import matplotlib.pyplot as plt

from pykalman import KalmanFilter
from openbb_terminal.sdk import openbb
# Now get stock price data using the OpenBB SDK.

data = openbb.stocks.load("LMT", start_date="2013-01-01", end_date="2015-01-01")
prices = data["Adj Close"]

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

ImportError: numpy.core.multiarray failed to import

In [5]:
# Step 2: Build a smoothed average with the Kalman filter

# Start by setting the initial paramaters for the Kalman filter.

kf = KalmanFilter(
    transition_matrices = [1],
    observation_matrices = [1],
    initial_state_mean = 0,
    initial_state_covariance = 1,
    observation_covariance=1,
    transition_covariance=0.01
)

In [6]:
# The transition matrix tells the algorithm how the system evolves from one state to another. The observation matrix tracks the next measurement given the predicted next state. The initial mean and covariance are the initial estimates of the state and error and the covariances measure the noise in the evolution of the algorithm. In more complex examples, you can calibrate these inputs to give a better first estimate.

# After you define the inputs for the Kalman filter algorithm, use it to filter the prices and find the “true” average.

state_means, _ = kf.filter(prices.values)
state_means = pd.Series(state_means.flatten(), index=prices.index)

mean30 = prices.rolling(window=30).mean()
plt.plot(state_means)
plt.plot(prices)
plt.plot(mean30)
plt.title('Kalman filter estimate of average')
plt.legend(['Kalman', 'Price', '30-day MA'])
plt.xlabel('Day')
plt.ylabel('Price')

NameError: name 'prices' is not defined

In [ ]:
# Zoom in on the last 200 days to get a closer look.

plt.plot(state_means[-200:])
plt.plot(prices[-200:])
plt.plot(mean30[-200:])
plt.title('Kalman filter estimate of average')
plt.legend(['Kalman', 'Price', '30-day MA'])
plt.xlabel('Day')
plt.ylabe('Price')